# Sample code for performing obstacle avoidance #

Import necessary packages.

In [1]:
# Code adapted from: https://github.com/bitcraze/crazyflie-lib-python/blob/master/examples/autonomousSequence.py

import time
import numpy as np
import cv2
import matplotlib.pyplot as plt

# CrazyFlie imports:

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander

Set your group number and camera number.

In [2]:
group_number = 14

# Possibly try 0, 1, 2 ...S
camera = 1

## Tune the red filtering ##

You can use the following cell to test and visualize the red filtering. This cell does *not* make the drone fly. It will connect to the CrazyFlie camera and perform red filtering on the live video feed. You should use this cell to tune the HSV intervals, and then copy/paste your tuned intervals into the __check_contours__ function below. When tuning the intervals, keep in mind that the lighting in the environment can matter.

In [22]:
cap = cv2.VideoCapture(camera)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()


    # Display the resulting frame
    cv2.imshow('frame', frame)

    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()

## Helper functions ##

The following cell contains some sample functions which will be useful.

In particular, __check_contours__ and __findGreatesContour__ will perform red filtering on the live camera feed and identify the obstacles. The red filtering is controlled by setting HSV intervals in the __check_contours__ function. Note that the intervals will require tuning and may vary on different drones/cameras.

The __adjust_position__ function can also be modified for performing obstacle avoidance.

In [3]:
# Get the current crazyflie position:
def position_estimate(scf):
    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]
            x = data['kalman.varPX']
            y = data['kalman.varPY']
            z = data['kalman.varPZ']
            
    print(x, y, z)
    return x, y, z


# Sort through contours in the image
def findGreatesContour(contours):
    largest_area = 0
    largest_contour_index = -1
    i = 0
    total_contours = len(contours)

    while i < total_contours:
        area = cv2.contourArea(contours[i])
        if area > largest_area:
            largest_area = area
            largest_contour_index = i
        i += 1

    #print(largest_area)

    return largest_area, largest_contour_index

def findContourCenter(contour):
    
    M = cv2.moments(contour)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])

    return cX, cY

# Find contours in the image
def check_contours(frame):

    #print('Checking image:')

    contour_boxes = []

    # These define the upper and lower HSV for the red obstacles.
    # Note that the red color wraps around 180, so there are two intervals.
    # Tuning of these values will vary depending on the camera.
    lb1 = (145, 35, 75)
    ub1 = (180, 255, 255)
    lb2 = (0, 75, 75)
    ub2 = (20, 255, 255)

    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)
    mask2 = cv2.inRange(hsv2, lb2, ub2)
    # Combine the masks.
    mask = cv2.bitwise_or(mask1, mask2)

    # Use the OpenCV findContours function.
    # Note that there are three outputs, but we discard the first one.
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    largest_area, largest_contour_index = findGreatesContour(contours)
  
    #x,y,w,h = cv2.boundingRect(contours[largest_contour_index])
    #cv2.rectangle(frame, (x,y), (x+w,y+h),(0,255,0), 2)
    
    
    if largest_area > 100:
        return True
    else:
        return False


# finds center of greatest contour in the image
def check_contours_centers(frame):

    #print('Checking image:')

    # These define the upper and lower HSV for the red obstacles.
    # Note that the red color wraps around 180, so there are two intervals.
    # Tuning of these values will vary depending on the camera.
    lb1 = (145, 35, 75)
    ub1 = (180, 255, 255)
    lb2 = (0, 75, 75)
    ub2 = (20, 255, 255)

    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)
    mask2 = cv2.inRange(hsv2, lb2, ub2)
    # Combine the masks.
    mask = cv2.bitwise_or(mask1, mask2)

    # Use the OpenCV findContours function.
    # Note that there are three outputs, but we discard the first one.
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    largest_area, largest_contour_index = findGreatesContour(contours)

    for contour in contours:
        
        x, y = findContourCenter(contours[largest_contour_index])

    return x,y

# Follow the setpoint sequence trajectory:
def adjust_position(cf, current_y):

    print('Adjusting position')

    steps_per_meter = int(10)
    # Set the number here (the iterations of the for-loop) to the number of side steps.
    # You may choose to tune the number and size of the steps.
    for i in range(3): 
        current_y = current_y - 1.0/float(steps_per_meter)
        position = [0, current_y, 0.0, 0.0]

        print('Setting position {}'.format(position))
        for i in range(10):
            cf.commander.send_position_setpoint(position[0],
                                                position[1],
                                                position[2],
                                                position[3])
            time.sleep(0.1)

    cf.commander.send_stop_setpoint()
    # Make sure that the last packet leaves before the link is closed.
    # The message queue is not flushed before closing.
    time.sleep(0.1)
    return current_y


# Ascend and hover:
def ascend_and_hover(cf):
    # Ascend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, y / 10)
        time.sleep(0.1)
    # Hover at 1 meter:
    for _ in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(0.1)
    return

def hover(cf):
    print('Hovering')
    # Hover at 1 meter:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(0.1)
    return

def hover_forward(cf):
    print('Forward')
    # Hover at 1 meter:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0.1, 0, 0, 1)
        time.sleep(0.1)
    return
    
    
# Hover, descend, and stop all motion:
def hover_and_descend(cf):
    # Hover at 1 meter:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(0.1)
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 10)
        time.sleep(0.1)
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
    return

def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    
    wait_for_position_estimator(cf)
    time.sleep(0.1)    
    return

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander
from cflib.positioning.motion_commander import MotionCommander


def wait_for_position_estimator(scf):
    print('Waiting for estimator to find position...')

    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    var_y_history = [1000] * 10
    var_x_history = [1000] * 10
    var_z_history = [1000] * 10

    threshold = 0.001
    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]

            var_x_history.append(data['kalman.varPX'])
            var_x_history.pop(0)
            var_y_history.append(data['kalman.varPY'])
            var_y_history.pop(0)
            var_z_history.append(data['kalman.varPZ'])
            var_z_history.pop(0)

            min_x = min(var_x_history)
            max_x = max(var_x_history)
            min_y = min(var_y_history)
            max_y = max(var_y_history)
            min_z = min(var_z_history)
            max_z = max(var_z_history)

            print("{} {} {}".
                format(max_x - min_x, max_y - min_y, max_z - min_z))

            if (max_x - min_x) < threshold and (
                    max_y - min_y) < threshold and (
                    max_z - min_z) < threshold:
                break


In [14]:
cap = cv2.VideoCapture(camera)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # These define the upper and lower HSV for the red obstacles.
    # Note that the red color wraps around 180, so there are two intervals.
    # Tuning of these values will vary depending on the camera.
    
    lb1 = (145, 35, 75)
    ub1 = (180, 255, 255)
    lb2 = (0, 75, 75)
    ub2 = (20, 255, 255)

    
    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)
    mask2 = cv2.inRange(hsv2, lb2, ub2)
    # Combine the masks.
    mask = cv2.bitwise_or(mask1, mask2)

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    largest_area, largest_contour_index = findGreatesContour(contours)
  
    x,y,w,h = cv2.boundingRect(contours[largest_contour_index])
    cv2.rectangle(frame, (x,y), (x+w,y+h),(0,255,0), 2)
        
    # Compute
    #cv2.imshow('mask', mask)    
    
   
    cv2.imshow('frame', frame)

    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()

IndexError: tuple index out of range

In [4]:
def controller(cf, x, y, x_cur, y_cur):
    """
    
    cf: crazyflie instance
    box_x: x coordinate of the center of the bounding box in the image
    box_y: y coordinate of the center of the bounding box in the image
    box_width: width of the bounding box in the image
    box_height: height of the bounding box in the image
    x_cur: current x position
    y_cur: current y position
    
    Return True to indicate that the drone is close to the target and thus exit the loop to stop and descend, new x, new y
    Return False to indicate continuing to follow the target, new x, new y.
    
    """
    
    #obstacle threshold
    obstacle_threshold = 0.2
    
    #### TO DO: Fill below ####
    # Exit condition/method using size of the bounding box
    # We experimentally found 0.993 to be the value of the box height for a person
    # at a reasonably close distance to stop using the live-video feed cell 

    
    if x_cur >= 30:
        print("done!", closest[2])
        return True, x_cur, y_cur
    
    
    y_command = y_cur

    
    #### TO DO: Fill below ####
    # Determine the x and y velocity
    if(abs(x) < 20):
        print("object near")
        
        if x <= 0:
            y_command = y_cur + 0.15
    
        if x > 0:
            y_command = y_cur - 0.15

        cf.commander.send_position_setpoint(0, y_command, 1, 0)
    
    cf.commander.send_hover_setpoint(0.15, 0, 0, 1)

    return False, x_cur + 0.15, y_command
    
    

## Test obstacle avoidance on the CrazyFlie ##

The following cell *will* fly the drone. Place the CrazyFlie in front of an obstacle in the netted area for testing. This cell will perform object detection and avoidance using the red filtering defined in the helper functions above.

In [19]:
import cv2
import time
import numpy as np

#threshold from center for obstacles
obstacle_threshold = 0.2

# Set the URI the Crazyflie will connect to
group_number = 14
uri = f'radio://0/{group_number}/2M'

# Possibly try 0, 1, 2 ...
camera = 1

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    ## Ascend to hover; run the sequence; then descend from hover:
    # Use the CrazyFlie corresponding to team number:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Get the Crazyflie class instance:
        cf = scf.cf
        x_cur = 0.0
        y_cur = 0.0

        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0

        # capture the video
        cap = cv2.VideoCapture(camera)
        
        # get the video frames' width and height
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))

        # flag indicating whether to exit the main loop and then descend
        exit_loop = False

        # Ascend and hover a bit
        set_PID_controller(cf)
        ascend_and_hover(cf)
        time.sleep(1)
        
        # detect objects in each frame of the video
        while cap.isOpened():
            # Try to read image
            ret, frame = cap.read()
            
            if ret:
                
                if(check_contours(frame):

                    #finds x and y center of the greatest contour in the frame (i.e the closest)
                    x, y = check_contours_centers(frame)
                    
                    exit_loop, x_cur, y_cur = controller(cf, x, y, x_cur, y_cur)
                                        
                else:
                    hover_forward(cf)                  
                    
                    
            else:
                hover(cf)
    
                
        cap.release()
        
        # Descend and stop all motion:
        hover_and_descend(cf)
        
        cv2.destroyAllWindows()

Scanning interfaces for Crazyflies...
Crazyflies found:
radio://0/14/2M
radio://0/14/2M
radio://0/14/2M
radio://0/14/2M
radio://0/14/2M
radio://0/14/2M
Initializing PID Controller
Waiting for estimator to find position...
999.9999900951216 999.9999900942421 999.9997553092253
999.9999900951216 999.9999900942421 999.9997553092253
999.9999900951216 999.9999900942421 999.9997553092253
999.9999900951216 999.9999900942421 999.9997568184044
999.9999902191612 999.9999902187292 999.9997568184044
999.9999902191612 999.9999902187292 999.9997568184044
999.9999902191612 999.9999902187292 999.9997568184044
999.9999902191612 999.9999902187292 999.9997568184044
999.9999902191612 999.9999902187292 999.9997568184044
1.2841319403378293e-06 1.285114194615744e-06 4.368467489257455e-05
Forward
Forward
Forward
Forward
Forward


KeyboardInterrupt: 

In [18]:
cap.release()

In [18]:
import cv2
import time
import numpy as np

#threshold from center for obstacles
obstacle_threshold = 0.2

# Set the URI the Crazyflie will connect to
group_number = 14
uri = f'radio://0/{group_number}/2M'

# Possibly try 0, 1, 2 ...
camera = 1


# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

if len(available) == 0:
    print('No Crazyflies found, cannot run example')
    
else:
    ## Ascend to hover; run the sequence; then descend from hover:
    # Use the CrazyFlie corresponding to team number:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        x,y,z = position_estimate(scf)
        print(x)



Scanning interfaces for Crazyflies...
Crazyflies found:
radio://0/14/2M
radio://0/14/2M
radio://0/14/2M
radio://0/14/2M
radio://0/14/2M
radio://0/14/2M
radio://0/14/2M
Hi
hello


KeyboardInterrupt: 